In [127]:
"""
This could be markdown later. Include something about the note book in here if necessary.
"""

'\nThis could be markdown later. Include something about the note book in here if necessary.\n'

In [144]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pendulum

In [118]:
"""
Data Wrangling Section 1: Assessment Page Views.csv
"""
#Read in csv as dataframe
#Using a smaller subset of the data since testing with 6mil+ rows would be too intensive
df = pd.read_csv('/Users/ethanrowe/Dev/springboard_data/KSU Assessment Page Views Fall 2018.csv', nrows = 10000)

In [48]:
#Eliminate Outliers using the 1.5*interquartile rule
IQR = float(df['duration'].quantile([0.75])) - float(df['duration'].quantile([0.25]))
upper_bound = float(df['duration'].quantile([0.75])) + 1.5*IQR
lower_bound = float(df['duration'].quantile([0.25])) - 1.5*IQR

df_clean = df[(df['duration'] < upper_bound) & (df['duration'] > lower_bound)]

In [114]:
print("Cleaned Data:")
print(df_clean.shape)
print("Null Values: " + str(df_clean.isnull().values.any()))
#df_clean.to_csv('KSU Assessment Page Views Fall 2018 Clean.csv')

Cleaned Data:
(8846, 13)
Null Values: False


In [52]:
#Determine if there are missing values missing values
total =  []
for chunk in pd.read_csv('/Users/ethanrowe/Dev/springboard_data/KSU Assessment Page Views Fall 2018.csv', chunksize = 1000):
    if chunk.isnull().values.any():
        total.append(chunk)
        
#There are missing values in the whole dataframe
#Need to check user_id, canvas_assignment_d, canvas_attempt_id, page_ids
#There are no missing values in the columns that we are taking an interest in for any of our calculations
#Checked each column for missing values and each column had none

In [110]:
#Group by page and by students to compare average assignment durations
assignment_df = pd.DataFrame(df_clean.groupby(['canvas_assignment_id'])['duration'].mean())
print('\nGrouped by Assignment')
assignment_df.columns = ['Average Duration']
print(page_df.shape)

user_df = pd.DataFrame(df_clean.groupby(['user_param_external_user_id', 'canvas_assignment_id'])['duration'].mean())
print('\nGrouped by User')
user_df.columns = ['Average Duration']
print(user_df.shape)


#page_df.to_csv('KSU Assessment Assignment Duration Averages.csv')
#user_df.to_csv('KSU Assessment Student Views by User.csv')    


Grouped by Assignment
(580, 1)

Grouped by User
(1865, 1)


In [111]:
#User_df is good for visualizing the data, but a df that is easier to work with will be labeled sub_df. It has the 
#same columns as user_df, just in a different order with a numerical index. It will also be sorted by 
#canvas_assignment_id in order to match assignment_df's index
sub_df = user_df.reset_index()  
sub_df = sub_df.set_index('canvas_assignment_id')  
sub_df = sub_df.sort_index()     
sub_df = sub_df.reset_index()    

#Using the user_df, turn it back into a dictionary grouped by canvas_assignment ids
dfs = {}       
for entry in sub_df.iterrows():  
    if index != entry[1]['canvas_assignment_id']:  
        index = entry[1]['canvas_assignment_id']  
        df = pd.DataFrame(columns=['canvas_assignment_id', 'user_param_external_user_id', 'Average Duration'])  
        df = df.append(dict(entry[1]), ignore_index = True) 
        dfs[index] = df  
    else:  
        dfs[index] = dfs[index].append(dict(entry[1]), ignore_index = True)  

#Now page_df and this dictionary dfs should have one to one matches for the loop below
#This loop will find all the student-assignment_id pairs that had an above average duration for that assignment
above_avg_df = pd.DataFrame(columns = ['canvas_assignment_id', 'user_param_external_user_id','Average Duration']) 
for entry in assignment_df.iterrows(): 
    df = dfs[entry[0]] 
    above_avg_chunk = df[df['Average Duration'] >= entry[1][0]] 
    above_avg_df = above_avg_df.append(above_avg_chunk) 
#above_avg_df.to_csv('KSU Students with Above Average Assignment Duration.csv')    

In [116]:
print("Students with longer than average assignmnet durations:")
print(above_avg_df.shape)
print(above_avg_df.head())

Students with longer than average assignmnet durations:
(1018, 3)
   canvas_assignment_id  user_param_external_user_id  Average Duration
0               40876.0                      37827.0      12831.500000
0               40878.0                      37949.0       9549.583333
3               40878.0                      37251.0      10307.500000
0               40879.0                      38399.0      11579.000000
5               40879.0                      40394.0      16064.857143


In [315]:
"""
Data Wrangling Section 2: Course Grades 
Course Grades has been cleand by hand in excel. The details are outlined in Data Wrangling.md
"""
#Import data
df2 = pd.read_csv('/Users/ethanrowe/Dev/springboard_data/Course Grades (Final Grade Only View) Fall 2018 - Course Grades (Final Grade Only View) Fall 2018.csv')
login_times_clean = pd.read_csv('/Users/ethanrowe/Dev/springboard_data/anon_students_login_times_clean.csv', index_col = 0)


In [297]:
#The next step will be to merge login_times_clean and df2. df2 has all of the grade 
#information for certain students by name and id

#I might need to use a different dataset since that one is not anonymous, check Course Final Grades or something later
#login_times_clean is also not anonymous and it has the last access time for students by name and id

#I want a dataframe that has students grades along with last access times. Then I can break that up into those who
#were active before and after certain periods to get a good idea of the distribution.

#All of the students in login_times_clean have final grades less than 70%. I can use df2 for a number of things on its
#own but I also need it to put login_times_clean into perspective.

#First step is to clean df2 if necessary. Then start prepping all of them. They should be EDA ready by the end of this.

"""
To-Do
Load in correct csv's, double check what I need for this
Merge them or something like that or augment login_times_clean with the necessary columns

Do stuff on df2 that is necessary for cleaning like checking for missing values.

Do stuff on login_times_clean such as breaking it up by times and counting stuff. Basically all of this has been 
done in excel already, but this will be more accurate and adaptable moving forward.

"""

"\nTo-Do\nLoad in correct csv's, double check what I need for this\nMerge them or something like that or augment login_times_clean with the necessary columns\n\nDo stuff on df2 that is necessary for cleaning like checking for missing values.\n\nDo stuff on login_times_clean such as breaking it up by times and counting stuff. Basically all of this has been \ndone in excel already, but this will be more accurate and adaptable moving forward.\n\n"

In [316]:
#df2 is the same as Course Grades (Final Grade Only View) Fall 2018 but with a column for student names
print(df2.head())

login_times_clean = login_times_clean.sort_index()
print(login_times_clean.head())


   student id  course id  course sis        section  section id  section sis  \
0       38651        626         NaN   WELL 1000/08        1165          NaN   
1       37597        655         NaN  WELL 1000/W83        1221          NaN   
2       35615        646         NaN   WELL 1000/38        1205          NaN   
3       37982        633         NaN  WELL 1000/W65        1179          NaN   
4       40291        623         NaN   WELL 1000/07        1164          NaN   

        term  term id  term sis  current score enrollment state  
0  Fall 2018       20       NaN          10.06           active  
1  Fall 2018       20       NaN          10.17           active  
2  Fall 2018       20       NaN          10.29           active  
3  Fall 2018       20       NaN          10.30           active  
4  Fall 2018       20       NaN          10.32           active  
   Course Id  Unposted Final Scores         Login Time  student id
0        639                    0.0  Aug 13 at 10:46pm  

In [ ]:
#Theses cell will not run out of order from the rest of the cells since they run permanent changes and use references.

#Let's clean this right now
login_times_clean.columns = ['course id', 'unposted final score',
       'login time', 'student id']

#first convert the login column into something that is readable to pd.to_datetime
for index, entry in enumerate(login_times_clean['login time']):
    login_times_clean['login time'][index] = entry[0:-2] + entry[-2:].upper() + ' 2018'
    if login_times_clean['login time'][index][0:4] == 'Sept':
        login_times_clean['login time'][index] = 'Sep ' + entry[4:]

In [318]:
#convert login time new into a datetime object
time_format = '%b %d at %H:%M%p %Y'
login_times_clean['login time'] = pd.to_datetime(login_times_clean['login time'], format=time_format)  

In [321]:
#Now this data frame is prepped for merging
print(login_times_clean.shape)
login_times_clean.head()

(490, 4)


,course id,unposted final score,login time,student id
0,639,0.0,2018-08-13 10:46:00,38094
1,659,0.0,2018-08-13 11:04:00,22168
2,659,0.0,2018-08-13 03:19:00,37144
3,638,0.0,2018-08-13 09:04:00,38000
4,631,0.0,2018-08-14 01:12:00,38410


In [325]:
#Merge
grades_login_times = pd.merge(login_times_clean, df2, on = ['student id', 'course id'], how = 'left')

#Now this is EDA ready
grades_login_times
grades_login_times.to_csv('grades_and_login_times(below 70).csv')

In [120]:
"""
Other Data sets to consider are Click History and Assessment Responses
"""

'\nOther Data sets to consider are Click History and Assessment Responses\n'